### 크롤링

In [1]:
# 크롤러 라이브러리
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
# 엑셀 라이브러리
import openpyxl
# 시간 라이브러리
import time
import pandas as pd

In [2]:
# 크롤링한 데이터를 모아두는 리스트
earned_content = []

# 캠퍼스픽 공모전 페이지와 대외활동 페이지 URL
contest_url = "https://www.campuspick.com/contest"
activity_url = "https://www.campuspick.com/activity"
url_list = [contest_url, activity_url]

In [ ]:
# 크롬 드라이버 초기화
driver = webdriver.Chrome()

# 모든 URL의 결과를 저장할 리스트
all_results = []

for url in url_list:
    driver.get(url)
    time.sleep(3)  # 로딩 대기

    # 무한 스크롤에서 첫 번째 리스트 아이템 가져오기
    lis = driver.find_elements(By.XPATH, '//*[@class="list"]/*[@class="item"]')
    li = lis[0] if lis else None
    count = 0  # 카운트 초기화

    while li and count < 70:  # 최대 70개 가져오기
        # 마감기한 확인
        if li.find_elements(By.XPATH, './/*[@class="dday highlight"]'):
            due_date = li.find_element(By.XPATH, './/*[@class="dday highlight"]').text
            if "마감" in due_date:
                print("마감된 게시물입니다.")
        else:
            due_date = "마감기한 없음"  # 마감기한이 없는 경우 처리

        # 키워드, 제목, 주최, 조회수, 썸네일 이미지 URL 가져오기
        keywords = li.find_elements(By.XPATH, './/*[@class="badges"]/span')
        keyword_list = [keyword.text for keyword in keywords]
        keyword_str = ",".join(keyword_list)

        title = li.find_element(By.XPATH, './/h2').text
        company = li.find_element(By.XPATH, './/*[@class="company"]').text
        view_count = li.find_element(By.XPATH, './/*[@class="viewcount"]').text
        thumbnail_url = li.find_element(By.XPATH, './/figure').get_attribute("data-image")

        results = [keyword_str, title, company, due_date, view_count, thumbnail_url]
        all_results.append(results)  # 모든 결과 리스트에 추가

        count += 1  # 카운트 증가
        print(f"{count}번째 결과: {results}")

        # 다음 리스트 아이템으로 스크롤
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", li)

        # 스크롤 후 대기 시간
        time.sleep(5)  # 5초 대기 (필요에 따라 조정 가능)

        # 다음 아이템 확인
        if li.find_elements(By.XPATH, './following-sibling::*[@class="item"]'):
            next_li = li.find_element(By.XPATH, './following-sibling::*[@class="item"]')
            li = next_li
        else:
            print("다음 리스트가 존재하지 않습니다. 종료합니다.")
            break

    # URL별로 결과 저장하기
    df = pd.DataFrame(all_results, columns=["키워드", "제목", "주최", "마감기한", "조회수", "썸네일"])
    df.to_csv(f'crawled_data_{url.split("/")[-1]}.csv', index=False, encoding='utf-8-sig')
    print(f'{url.split("/")[-1]}에 대한 결과를 저장했습니다.')

    # URL별 결과 초기화
    all_results = []

# 크롬 드라이버 종료
driver.quit()



### 데이터 확인

In [8]:
import pandas as pd

In [9]:
df_activity = pd.read_csv('C:\\Users\\User\\Desktop\\backend\\crawl\\contest_supporter_campuspick\\crawled_data_activity.csv')
df_contest = pd.read_csv('C:\\Users\\User\\Desktop\\backend\\crawl\\contest_supporter_campuspick\\crawled_data_contest.csv')

In [11]:
# 각 DataFrame에 parentCategory_id와 subCategory_id 열 추가
#df_activity['parentCategory_id'] = 1  
#df_activity['subCategory_id'] = 2    

#df_contest['parentCategory_id'] = 1   
#df_contest['subCategory_id'] = 4   

In [ ]:
# 수정된 DataFrame을 동일한 파일명으로 저장 (덮어쓰기)
#df_activity.to_csv('C:\\Users\\User\\Desktop\\backend\\crawl\\contest&supporter_campuspick\\crawled_data_activity.csv', index=False, encoding='utf-8-sig')
#df_contest.to_csv('C:\\Users\\User\\Desktop\\backend\\crawl\\contest&supporter_campuspick\\crawled_data_contest.csv', index=False, encoding='utf-8-sig')

### Activity data type convert

In [15]:
# 조회수 열을 정수형으로 변환
df_activity['조회수'] = df_activity['조회수'].fillna(0).astype(int)

In [21]:
df_activity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   키워드                70 non-null     object
 1   제목                 70 non-null     object
 2   주최                 70 non-null     object
 3   마감기한               70 non-null     object
 4   조회수                70 non-null     int32 
 5   썸네일                70 non-null     object
 6   parentCategory_id  70 non-null     int64 
 7   subCategory_id     70 non-null     int64 
dtypes: int32(1), int64(2), object(5)
memory usage: 4.2+ KB


In [20]:
df_activity.to_csv('C:\\Users\\User\\Desktop\\backend\\crawl\\contest_supporter_campuspick\\crawled_data_activity.csv', index=False, encoding='utf-8-sig')